In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from dataset_prepare import ToStrLower, DropDuplicates, FilterByQuantile
from autocompleter import AutoCompleterBaseline, AutoCompleteBySuffix, AutoCompleteByInvertedIndex
from inverted_index import InvertedIndex
from tries import make_trie

In [2]:
popular_queries = pd.read_csv("data/query_popularity.csv")
popular_queries['query'] = popular_queries['query'].astype(str).str.lower()

In [208]:
columns = ['wbuser_id', 'UQ', 'cnt', 'locale', 'weekday', 'time']
search_history = pd.read_csv("data/search_subset.csv")[columns]

# Filter data

In [5]:
CNT_UPPER_QUANTILE = 0.9

In [6]:
# to lower UQ
to_str_lower_filter = ToStrLower(column="UQ")
df = to_str_lower_filter.apply(search_history)

In [7]:
# filter out duplicates by wbuser_id 
subset_drop = ["wbuser_id", "UQ"]
drop_duplicates_filter = DropDuplicates(subset_drop)
df = drop_duplicates_filter.apply(df)

DropDuplicates: dataset size 499999 -> 450085


In [8]:
UPPER_QUANTILE = 0.95
LOWER_QUANTILE = 0
quantile_filter = FilterByQuantile('cnt', UPPER_QUANTILE)
df = quantile_filter.apply(df)

Lower quantile: 0. Upper quantile: 33128.79999999999
FilterByQuantile: dataset size 450085 -> 427580


### Autocompleters

#### Baseline

In [70]:
# baseline
autocompleter_baseline = AutoCompleterBaseline(query_database=popular_queries, max_n=10)
autocompleter_baseline.predict("игровой")

,query,query_popularity
652,игровой ноутбук,10
694,игровой ноут,10
5030,игровой компьютер,10
6583,игровой руль,10
8203,игровой комплекс для кошек,10
15001,игровой компьютерный стол,10
23432,игровой комплекс для кошки,10
30519,игровой столик,10
33981,игровой центр батут,9
38301,игровой набор кухня детский,9


In [73]:
autocompleter_baseline.predict("игровой ноут")

Found for word: игровой 169 instances
Found for word: ноут 411 instances


,query,query_popularity
268,ноутбук игровой,10
652,игровой ноутбук,10
694,игровой ноут,10


#### Tries

In [9]:
autocompleter_tries = AutoCompleteBySuffix()
autocompleter_tries.make_trie(popular_queries)

Making database for queries... It length: 336988


In [10]:
autocompleter_tries.query("игровой")[:10]

[('игровой ноут', 1, 10),
 ('игровой ноутбук', 1, 10),
 ('игровой набор', 2, 9),
 ('игровой набор кухня детский', 1, 9),
 ('игровой набор для девочек', 1, 9),
 ('игровой набор для сада', 1, 9),
 ('игровой набор для мальчиков 7 лет', 1, 5),
 ('игровой набор доктор', 1, 7),
 ('игровой набор доктора с халатом', 1, 7),
 ('игровой набор доктора для девочек', 1, 5)]

In [15]:
sorted(autocompleter_tries.query("игровой"), key=lambda x: x[2], reverse=True)[:10]

[('игровой ноут', 1, 10),
 ('игровой ноутбук', 1, 10),
 ('игровой компьютер', 1, 10),
 ('игровой компьютерный стол', 1, 10),
 ('игровой комплекс для кошек', 1, 10),
 ('игровой комплекс для кошки', 1, 10),
 ('игровой руль', 1, 10),
 ('игровой столик', 1, 10),
 ('игровой набор', 2, 9),
 ('игровой набор кухня детский', 1, 9)]

### Tries and inverted index

In [3]:
# inverted index

query_popularity_corpus = zip(popular_queries['query'], popular_queries['query_popularity'])
inverted_index = InvertedIndex()
index = inverted_index.process_corpus(query_popularity_corpus)

In [4]:
# tries 
trie = make_trie(popular_queries)

Making database for queries... It length: 336988


In [36]:
autocomplete_by_inverted_index = AutoCompleteByInvertedIndex(index, trie, first_prefix=5, max_candidates=50)

In [40]:
# игра на -> игра настольная
query = "ноутбук"
result = autocomplete_by_inverted_index.query(query, max_n=10)

In [42]:
result

[Query indx: 0. Words: ноутбук. Score: 10,
 Query indx: 3120. Words: ноутбук 17 ca2033ur. Score: 10,
 Query indx: 3154. Words: ноутбук 1650. Score: 10,
 Query indx: 3160. Words: ноутбук apple. Score: 10,
 Query indx: 2151. Words: ноутбук. Score: 10,
 Query indx: 20511. Words: сумка для ноутбука 13.3. Score: 10,
 Query indx: 2160. Words: acer / ноутбук acer extensa ex215-52-54ne/i5-1035g1/8gb/512gb ssd/15.6\x22 fhd/uma/noos. Score: 10,
 Query indx: 30835. Words: sun voyage ноутбук. Score: 10,
 Query indx: 30840. Words: рюкзак мужской для ноутбука. Score: 10,
 Query indx: 5278. Words: ноутбук леново. Score: 10]

In [41]:
autocomplete_by_inverted_index.get_candidates("ноу")

[('ноутбука', 218, 0),
 ('ноут', 96, 0),
 ('ноутбук', 75, 0),
 ('ноутбуков', 4, 0),
 ('ноутбуки', 3, 0),
 ('ноутбука,', 2, 0),
 ('ноутбуку', 2, 0),
 ('ноута', 2, 0),
 ('ноу', 1, 0),
 ('ноутбукаи', 1, 0),
 ('ноутбуком', 1, 0),
 ('ноутбук-алфавит', 1, 0),
 ('ноутбку', 1, 0),
 ('ноутбкука', 1, 0),
 ('ноутука', 1, 0),
 ('ноут9', 1, 0),
 ('ноут5', 1, 0),
 ('ноубук', 1, 0),
 ('ноуи', 1, 0)]